<a href="https://colab.research.google.com/github/dgromann/ProgrammingForTranslators/blob/master/tutorial/4_Beispiel_Termextraktion.ipynb" target="_parent"></a>

# Programmieren für ÜbersetzerInnen - Beispiel Termextraktion




Als erstes Beispiel nach der Einführung schreiben wir ein simples Programm zur Termextraktion unter Verwendung des statistischen Maßes TF-IDF. TF-IDF steht für Termhäufigkeit (TF) und Inverse Dokumentenhäufigkeit (IDF) und ist ein Maß zur Berechnung der Gewichtung eines Wortes/einer Phrase in einem Dokument -  damit können fachsprachliche Benennungen extrahiert werden.

Unser Beispielkorpus besteht aus den folgenden drei Sätzen: 


```
document1 = "Sendung von Gold in einem Container." 
document2= "Lieferung von Silber in einem silbernen LKW angekommen." 
document3 = "Sendung von Gold in einem LKW angekommen."
```
Anstatt diese Sätze direkt im Notebook zu schreiben, laden wir drei Dokumente in unser Programm:


In [1]:
# Datein hochladen
corpus = []

document1 = open("document1.txt")
document2= open("document2.txt")
document3 = open("document3.txt")

documents = [document1, document2, document3]

# Die Funktion strip() dient dazu das Symbol \n für die Markierung des Zeilenendes  
# und alle zusätzlichen Leerzeichen vor oder nach der Zeile zu entfernen
for document in documents:
  for sentence in document.readlines():
    corpus.append(sentence.strip())

#Wie Sie sehen wird für jedes Dokument eine Liste erstellt, die jede Zeile (in diesem Fall nur eine), des Dokuments enthält.
print(corpus)

['Sendung von Gold in einem Container.', 'Lieferung von Silber in einem silbernen LKW angekommen.', 'Sendung von Gold in einem LKW angekommen.']


Um spaCy auch in diesem Notebook verwenden zu können, müssen wir spaCy und auch das deutsche Sprachpaket wieder laden:

In [2]:
# spaCy in Colab/Binder installieren - für Anaconda siehe spacy Installatiosnanweisungen
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Requirement already up-to-date: spacy in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.2.3)
    100% |████████████████████████████████| 12.0MB 84.1MB/s a 0:00:0101
  Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
  Running setup.py install for en-core-web-sm ... done
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
    100% |████████████████████████████████| 14.9MB 105.2MB/s ta 0:00:01
  Running setup.py install for de-core-news-sm ... done
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


Wir verwenden hier die Vorverarbeitungsmethode aus dem letzten Beispiel. Da diese in einem anderen Notebook steht, müssen wir Sie hier noch einmal wiedergeben: 

In [3]:
import spacy
nlp = spacy.load('de_core_news_sm')

def preprocessing(sentence):
  sentence = sentence.lower() 
  # Wir fügen auch gleich ein paar weitere POS-Tags für Konjunktionen, etc. hinzu 
  pos_to_be_removed =['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
  text_out = []
  # Tokenisiert und weitere Vorverarbeitung
  doc = nlp(sentence)
  for token in doc:
    # POS-Tags überprüft und nur jene die nicht in "pos_to_be_removed" zu finden sind berücksichtigen
    if token.pos_ not in pos_to_be_removed :
      #Lemmatisierung
      lemma = token.lemma_
      text_out.append(lemma)  
  return text_out

**Aufgabe:** <br>
Rufen Sie die oben definierte Methode `preprocessing` für jeden Satz des Korpus `corpus` auf und speichern Sie das Ergebnis in eine neue Liste `preprocessed`:

In [4]:
# Neue Liste preprocessed 
preprocessed = []

# Schreiben Sie hier Ihre Code - bedenken Sie dass corpus eine Liste ist 
for sentence in corpus:
  prep_sentence = preprocessing(sentence)
  preprocessed.append(prep_sentence)

# Das Resultat ist eine eigene Liste an Worte für jedes Dokument
# Die nachstehende Zeile fügt die einzelnen Worte für jedes Dokument wieder zu Sätzen zusammen
preprocessed = [" ".join(x) for x in preprocessed] 
print(preprocessed)

['sendung gold container', 'lieferung silber silbern lkw ankommen', 'sendung gold lkw ankommen']


# TF-IDF
Anstelle der manuellen Berechnung der TF-IDF Werte, verwenden wir eine bereitgestellte Library für maschinelles Lernen names `sklearn`. 



In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed)
word_index = vectorizer.vocabulary_

print("Wortindex: ", word_index)
print("TF-IDF Matrix: ")
print("(Dokumentennummer, Termindex) TF-IDF Weight")
print(tfidf_matrix)

Wortindex:  {'sendung': 5, 'gold': 2, 'container': 1, 'lieferung': 3, 'silber': 6, 'silbern': 7, 'lkw': 4, 'ankommen': 0}
TF-IDF Matrix: 
(Dokumentennummer, Termindex) TF-IDF Weight
  (0, 5)	0.5178561161676974
  (0, 2)	0.5178561161676974
  (0, 1)	0.680918560398684
  (1, 3)	0.49047908420610337
  (1, 6)	0.49047908420610337
  (1, 7)	0.49047908420610337
  (1, 4)	0.3730219858594306
  (1, 0)	0.3730219858594306
  (2, 5)	0.5
  (2, 2)	0.5
  (2, 4)	0.5
  (2, 0)	0.5


Der nachstehende Code extrahiert die `n` wichtigsten Terme aus unserem Mini-Korpus. 



**Frage:** <br>
Würden Sie sagen, dass diese Auswahl für diesen Mini-Korpus Sinn macht? Stellen diese Worte tatsächlich wichtige semantische Bestandteile unseres Korpus dar?

In [6]:
import numpy as np

feature_array = np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]
print(top_n)

['sendung' 'lkw' 'gold']


Um die wichtigsten extrahieren Termini in einer Datei speichern zu können gehen wir wie folgt vor: 

In [7]:
# Speichern Sie die resultierende Datei in Ihrem lokalen Dateiverzeichnis
with open('results.txt', 'w') as f:
    for item in top_n:
        f.write("%s\n" % item)